In [ ]:
import os
import re
import time
import openpyxl
import pandas as pd
import pdfplumber
import nltk
from nltk.tokenize import word_tokenize

report_path = "./Report_crawler/Report_set"
text_path = ".//Report_crawler/Report_text"

def help() :
    print("createFolder : create folder on path")
    print("pdfTextReader : return extracted text on pdf")
    print("tokenize : return tokenized text")
    print("processing : pre-processing tokenized text")

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def pdfTextReader(file) :
    content = ""
    try :
        with pdfplumber.open(file) as pdf_file :
            for page in pdf_file.pages :
                try :
                    for text in page.extract_text() :
                        content += text
                except Exception as e:
                    continue
    except Exception as e :
        print(e)
    return content

def tokenize(text) :
    text = re.sub("[^0-9a-zA-Z가-힣]", " ", text)
    tokenized_text = word_tokenize(text)
    return tokenized_text

def processing(text_list) :
    stopword_list = pd.read_table("./한글불용어.txt", sep = "\n")

    for text in text_list :
        if len(text) <= 1 :
            text_list.remove(text)
        for stopword in stopword_list['불용어'] :
            if stopword in text :
                text.replace(stopword, "")

    return text_list



if __name__ == "__main__" :
    start_time = time.time()

    createFolder(text_path)

    wb = openpyxl.Workbook()
    sheet = wb.active

    num = 0
    for company in os.listdir(report_path) :
        corp = os.listdir(report_path + "/" + company)
        for report in corp :
            report_pdf = open(report_path + "/" + company + "/" + report, 'rb')
            text = pdfTextReader(report_pdf)
            tokenized_text = tokenize(text)

            num += 1
            data = [company, report, ' '.join(tokenized_text)]
            for index, dt in enumerate(data) :
                sheet.cell(row = num, column = index+1, value = dt)
            print(num)
    
    wb.save("./Report_crawler/Report_text/Report_Data.xlsx")
    print("실행 시간 : ", time.time() - start_time)

